<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

In [91]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from src.SupportPreProcesamiento import (
    exploracion_dataframe,
    separarar_df,
    plot_numericas,
    plot_categoricas,
    relacion_vr_categoricas,
    relacion_vr_numericas,
    matriz_correlacion,
    detectar_outliers,
    diferencia_tras_rellenar_nulos
)

pd.set_option('display.float_format', '{:.2f}'.format)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor


# Laboratorio EDA y Gestión de Nulos

AutoRenew, una empresa líder en la venta de coches de segunda mano, te ha contratado para desarrollar un modelo predictivo que estime el precio de los vehículos usados. Este modelo ayudará a la empresa a mejorar sus estrategias de ventas, marketing y optimización del inventario. Para ello, cuentas con un conjunto de datos que contiene información detallada sobre diversas características de los coches usados. Sin embargo, como es común en conjuntos de datos reales, encontrarás valores nulos y necesitarás realizar un análisis exploratorio para identificar patrones, correlaciones y posibles anomalías.

 En este laboratorio, tu tarea es realizar un análisis exploratorio de los datos (EDA) y gestionar los valores nulos que encuentres. Este es solo el primer paso de un proyecto que se desarrollará durante toda la semana, en el que usarás este mismo conjunto de datos para practicar diferentes fases del preprocesamiento.

**Columnas del conjunto de datos:**

- **dateCrawled**: Fecha y hora en que se recolectaron los datos del coche.

- **name**: Nombre del anuncio, generalmente incluye la marca y el modelo del coche.

- **seller**: Tipo de vendedor (particular o concesionario).

- **offerType**: Tipo de oferta (venta o subasta).

- **price**: Precio del coche en la moneda local.

- **abtest**: Campo utilizado para pruebas A/B, indicando diferentes versiones del sitio web o anuncio.

- **vehicleType**: Tipo de vehículo (sedán, SUV, hatchback, etc.).

- **yearOfRegistration**: Año de registro del coche.

- **gearbox**: Tipo de caja de cambios (manual o automática).

- **powerCV**: Potencia del coche en caballos de vapor (CV).

- **model**: Modelo específico del coche.

- **kilometer**: Kilometraje total del coche.

- **monthOfRegistration**: Mes de registro del coche.

- **fuelType**: Tipo de combustible (gasolina, diésel, eléctrico, etc.).

- **brand**: Marca del coche.

- **notRepairedDamage**: Indica si el coche tiene daños no reparados.

- **dateCreated**: Fecha de creación del anuncio.

- **nrOfPictures**: Número de fotos en el anuncio.

- **postalCode**: Código postal de la ubicación del coche.

- **lastSeen**: Última fecha y hora en que el anuncio fue visto.

**Instrucciones:**

1. **Análisis Exploratorio de Datos (EDA):** Explora el conjunto de datos para familiarizarte con las diferentes características de los coches. Utiliza gráficos y estadísticas descriptivas para identificar patrones, tendencias y posibles anomalías.
   
2. **Gestión de Valores Nulos:** Identifica las columnas que contienen valores nulos y aplica las técnicas adecuadas para manejarlos. Esto podría incluir la imputación de valores, eliminación de filas o columnas, o cualquier otra estrategia que consideres apropiada.

**Nota:** Este laboratorio es el primer paso de una serie de actividades que realizarás esta semana utilizando el mismo conjunto de datos. Asegúrate de documentar bien tu trabajo, ya que lo utilizarás como base para los siguientes ejercicios.


In [92]:
df = pd.read_csv("datos/autorenew.csv")

In [93]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerCV,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,private,offer,480,test,NaN,1993,manually,0.00,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,private,offer,18300,test,coupe,2011,manually,187.00,NaN,125000,5,diesel,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",private,offer,9800,test,suv,2004,automatic,161.00,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,private,offer,1500,test,small car,2001,manually,74.00,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,private,offer,3600,test,small car,2008,manually,68.00,fabia,90000,7,diesel,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


# EDA
Vamos a realizar lo siguiente mediante las funciones previamente definidas:
- explorar la info del Dataframe
- visualizar las numéricas con un histplot
- visualizar las categoricas con un countplot
- visualizar los outliers con un boxplot
- visualizar correlacion con heatmap
- visualizar categoricas vs variable respuesta(barplot)
- visualizar numericas vs variable respuesta con un (scatterplot)

### Explorar la info del Dataframe
Vamos a ver primero que datos tenemos y con que vamos a trabajar vamos a usar la funcion explorar DataFrame, usando como columna control


In [94]:
exploracion_dataframe(df,"price",estadisticos=False)

El número de datos es 371528 y el de columnas es 20

 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 4

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
vehicleType,36.00
gearbox,5.44
model,5.51
fuelType,8.99
notRepairedDamage,19.40



 ..................... 

Los tipos de las columnas son:


,tipo_dato
dateCrawled,object
name,object
seller,object
offerType,object
price,int64
abtest,object
vehicleType,object
yearOfRegistration,int64
gearbox,object
powerCV,float64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna DATECRAWLED tiene las siguientes valore únicos:


...

,count
notRepairedDamage,
no,263182
yes,36286


La columna DATECREATED tiene las siguientes valore únicos:


,count
dateCreated,
2016-04-03 00:00:00,14450
2016-04-04 00:00:00,14022
2016-03-20 00:00:00,13547
2016-03-12 00:00:00,13379
2016-03-21 00:00:00,13305


La columna LASTSEEN tiene las siguientes valore únicos:


,count
lastSeen,
2016-04-06 13:45:54,17
2016-04-07 06:45:59,17
2016-04-07 08:16:54,16
2016-04-06 09:46:00,16
2016-04-06 04:17:20,16


### Revisar tipo de nulos que tenemos

In [95]:
df[["vehicleType","gearbox","model","fuelType","notRepairedDamage"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   vehicleType        237765 non-null  object
 1   gearbox            351319 non-null  object
 2   model              351044 non-null  object
 3   fuelType           338142 non-null  object
 4   notRepairedDamage  299468 non-null  object
dtypes: object(5)
memory usage: 14.2+ MB


In [96]:
for col in df[["vehicleType","gearbox","model","fuelType","notRepairedDamage"]].columns:
    print("\n ..................... \n")
    print(f"{col}")
    display(df[col].unique())


 ..................... 

vehicleType


array([nan, 'coupe', 'suv', 'small car', 'convertible', 'bus', 'combi',
       'other'], dtype=object)


 ..................... 

gearbox


array(['manually', 'automatic', nan], dtype=object)


 ..................... 

model


array(['golf', nan, 'grand', 'fabia', '3er', '2_reihe', 'andere', 'c_max',
       '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'kangoo', 'corsa', 'one',
       'fortwo', '1er', 'b_klasse', 'signum', 'astra', 'a8', 'jetta',
       'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter', '156', 'escort',
       'forester', 'xc_reihe', 'scenic', 'a4', 'a1', 'insignia', 'combo',
       'focus', 'tt', 'a6', 'jazz', 'omega', 'slk', '7er', '80', '147',
       'glk', '100', 'z_reihe', 'sportage', 'sorento', 'v40', 'ibiza',
       'mustang', 'eos', 'touran', 'getz', 'a3', 'almera', 'megane',
       'lupo', 'r19', 'zafira', 'caddy', 'mondeo', 'cordoba', 'colt',
       'impreza', 'vectra', 'berlingo', 'm_klasse', 'tiguan', 'i_reihe',
       'espace', 'sharan', '6_reihe', 'panda', 'up', 'seicento', 'ceed',
       '5_reihe', 'yeti', 'octavia', 'mii', 'rx_reihe', '


 ..................... 

fuelType


array(['petrol', 'diesel', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electro'], dtype=object)


 ..................... 

notRepairedDamage


array([nan, 'yes', 'no'], dtype=object)

# Conclusiones 
- Tenemos 4 Duplicados
- Tenemos 5 Columnas con nulos:
    - vehicleType: 36% nulos categórica ['nan','coupe', 'suv', 'small car', 'convertible', 'bus', 'combi',
       'other']
    - gearbox: 5,5% nulos, categórica ['manually', 'automatic', nan]
    - model: 5,51% nulos, categórica (No la pongo que son muchos)
    - fuelType: 8,98% nulos categórica ['petrol', 'diesel', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electro']
    - notRepairedDamage 19,39% nulos categorica [nan, 'yes', 'no']



# Las gráficas tardan mucho, porque? veamos:

In [97]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,371528.00,17295.14,3587953.74,0.00,1150.00,2950.00,7200.00,2147483647.00
yearOfRegistration,371528.00,2004.58,92.87,1000.00,1999.00,2003.00,2008.00,9999.00
powerCV,371528.00,113.97,189.52,0.00,69.00,104.00,148.00,19726.00
kilometer,371528.00,125618.69,40112.34,5000.00,125000.00,150000.00,150000.00,150000.00
monthOfRegistration,371528.00,5.73,3.71,0.00,3.00,6.00,9.00,12.00
nrOfPictures,371528.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
postalCode,371528.00,50820.67,25799.08,1067.00,30459.00,49610.00,71546.00,99998.00


Si miramos el valor máximo, de las columnas, no tiene sentido ni para price, ni yearof, ni powerCV, ni postal code, pero sobre todo vamos a limpiar nuestro DF, number of pictures no nos interesa, si siempre es 0, no afectara

Postal code nos puede interesar

Dropeamos nrOfPictures

In [98]:
df.drop(columns="nrOfPictures",inplace=True)
df.sample()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerCV,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode,lastSeen
182917,2016-03-12 12:39:00,Mercedes_Benz_E_250_Diesel_5_Gang_Sammlerzustand,private,offer,2950,test,NaN,1991,manually,94.00,e_klasse,150000,11,diesel,mercedes_benz,no,2016-03-12 00:00:00,66740,2016-03-12 12:39:00


### Veamos ahora price

In [99]:
df["price"].nlargest(100).unique()

array([2147483647,   99999999,   99000000,   74185296,   32545461,
         27322222,   14000500,   12345678,   11111111,   10010011,
         10000000,    9999999,    3895000,    3890000,    2995000,
          2795000,    1600000,    1300000,    1250000,    1234566,
          1111111,    1010010,    1000000,     999999,     999990,
           911911,     849000,     820000,     780000,     745000,
           725000,     700000,     650000,     619000,     600000,
           599000,     585000,     579000,     517895])

Con precios tan elevados y totalmente surrealistas, vamos a poner como nulos aquellos mayores a 3.895.000, y vamos también a poner como nulos [999999, 999990,1010010,1234566,1111111,911911] la razón son precios anormales, posiblemente errores o manera del usuario de negociar el precio por privado, sea la razón que sea, el número de datos afectado es pequeño y nos permitirá obtener datos más realistas

In [100]:
cond1 = df["price"] > 3895000
df.loc[cond1, "price"] = np.nan

In [101]:
atipicos = [999999, 999990,1010010,1234566,1111111,911911]
for ati in atipicos:
    condicion = df["price"] == ati
    df.loc[condicion,"price"] == np.nan

In [102]:
df["price"].nlargest()

366861   3895000.00
364171   3890000.00
41980    2995000.00
45055    2795000.00
333699   1600000.00
Name: price, dtype: float64

y ahora cual es el precio más bajo? Si es menos de 150 es nulo el precio

In [103]:
condicion = df["price"] < 150
df.loc[condicion,"price"] = np.nan

Con esto ya tenemos un rango de precios más adecuado

### YearofRegistration
Con el año de registro es fácil, si es mayor de 2024, es nulo

In [104]:
cond = df["yearOfRegistration"] > 2024
df.loc[cond,"yearOfRegistration"] = np.nan


y claro, el coche se invento en 1886, si es menor de eso es NAN

In [105]:
cond = df["yearOfRegistration"] < 1886
df.loc[cond,"yearOfRegistration"] = np.nan

### Month
Verifiquemos que hay

In [106]:
df["monthOfRegistration"].unique()

array([ 0,  5,  8,  6,  7, 10, 12, 11,  2,  3,  1,  4,  9])

El mes no nos va a aportar nada de valor tampoco, no me lo voy a cargar, pero si que voy a no tenerlo en cuenta

### PowerCV
un coche no puede tener menos de 5 CV, ni más de 1600 CV, asi que todo lo que se salga de rango, lo dejamos en NAN

In [107]:
cond = df["powerCV"] > 1600
df.loc[cond,"powerCV"] = np.nan

In [108]:
cond = df["powerCV"] < 5
df.loc[cond,"powerCV"] = np.nan

# Postal code
Considero que no la vamos a usar, ademas considero que es una variable categórica

In [109]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,356539.00,6138.49,18281.53,150.00,1300.00,3150.00,7500.00,3895000.00
yearOfRegistration,371346.00,2003.35,7.78,1910.00,1999.00,2003.00,2008.00,2019.00
powerCV,330442.00,124.73,66.04,5.00,79.00,114.00,148.00,1578.00
kilometer,371528.00,125618.69,40112.34,5000.00,125000.00,150000.00,150000.00,150000.00
monthOfRegistration,371528.00,5.73,3.71,0.00,3.00,6.00,9.00,12.00
postalCode,371528.00,50820.67,25799.08,1067.00,30459.00,49610.00,71546.00,99998.00


# ¿Con que columnas numéricas vamos a trabajar?
- price
- yearOfRegistration
- powerCV

Considero que el resto de columnas no aportan valor 

# Columnas categóricas
Ahora vamos a ver con que columnas categóricas nos quedamos y como gestionamos sus nulos

In [115]:
df.select_dtypes(include="O").head()

,dateCrawled,name,seller,offerType,abtest,vehicleType,gearbox,model,fuelType,brand,notRepairedDamage,dateCreated,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,private,offer,test,NaN,manually,golf,petrol,volkswagen,NaN,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,private,offer,test,coupe,manually,NaN,diesel,audi,yes,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",private,offer,test,suv,automatic,grand,diesel,jeep,NaN,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,private,offer,test,small car,manually,golf,petrol,volkswagen,no,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,private,offer,test,small car,manually,fabia,diesel,skoda,no,2016-03-31 00:00:00,2016-04-06 10:17:21


In [116]:
df.select_dtypes(include="O").columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'vehicleType',
       'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage',
       'dateCreated', 'lastSeen'],
      dtype='object')

### Columnas categóricas que tienen que ver con el precio:
- vehicleType
- gearbox
- fuelType
- brand
- notRepairedDamage
- seller

De estás veamos si se pueden usar

In [118]:
df["vehicleType"].unique()

array([nan, 'coupe', 'suv', 'small car', 'convertible', 'bus', 'combi',
       'other'], dtype=object)

In [119]:
df["gearbox"].unique()

array(['manually', 'automatic', nan], dtype=object)

In [121]:
df["fuelType"].unique()

array(['petrol', 'diesel', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electro'], dtype=object)

In [122]:
df["brand"].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'porsche', 'toyota', 'chevrolet',
       'dacia', 'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar',
       'daewoo', 'rover', 'land_rover', 'lada'], dtype=object)

In [ ]:
df["notRepairedDamage"].unique()

array([nan, 'yes', 'no'], dtype=object)

In [125]:
df["seller"].unique()

array(['private', 'commercial'], dtype=object)

Podemos usarlas, vamos a crear un nuevo Dataframe con las columnas que usaremos, numericas y 

### Visualizar las numéricas con un histplot


In [111]:
numericas, categoricas = separarar_df(df)
plot_numericas(numericas)

Error in callback <function flush_figures at 0x0000020139458CC0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

### Visualizar las categoricas con un countplot


### Visualizar los outliers con un boxplot


### Visualizar correlacion con heatmap


### Visualizar categoricas vs variable respuesta(barplot)


### Visualizar numericas vs variable respuesta con un (scatterplot)